In [1]:
import numpy as np
from copy import deepcopy
import random

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

/home/hhchung/dyngraph-uda/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.nodeproppred.evaluate import Evaluator

In [5]:
import sys
sys.path.append("..")
from dataset import temp_partition_arxiv

In [6]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')

In [7]:
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Model Structure ##

In [8]:
class OneLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        return x

class TwoLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        return x

class ThreeLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv3(x, edge_index)
        return x
    

    
class MLPHead(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x = self.linear1(x)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        x = self.linear2(x)
        # x = F.softmax(x, dim=1)
        return x

In [9]:
def train(encoder, mlp, optimizer, data):
    encoder.train()
    mlp.train()
    
    out = F.log_softmax(mlp(encoder(data.x, data.edge_index)), dim=1)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()
    
@torch.no_grad()
def test(encoder, mlp, data, evaluator):
    encoder.eval()
    mlp.eval()
    
    out = F.log_softmax(mlp(encoder(data.x, data.edge_index)), dim=1)
    val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask]).item()
    y_pred = out.argmax(dim=-1, keepdim=True)
    val_acc = evaluator.eval({
        'y_true': data.y[data.val_mask].unsqueeze(1),
        'y_pred': y_pred[data.val_mask],
    })['acc']
    
    return val_loss, val_acc

## Load Data ##

In [10]:
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')
data = dataset[0]
data.edge_index = to_undirected(data.edge_index, data.num_nodes) # mimicking barlow twins repo

## Data Partition ##

* Train: 0-2011
* Val: 2012
* Test:
** 2013-2014 (then adapt 2012-2013 adapt-val 2014)
** 2015-2016 (then adapt 2014-2015 adapt-val 2016)
** 2017-2018 (then adapt 2016-2017 adapt-val 2018)
** 2019-2020

## Source Training Stage ##

* Train: 0-2011
* Val: 2012

In [11]:
feat_dim = data.x.shape[1]
class_dim = data.y
hidden_dim = 128
emb_dim = 256
encoder = TwoLayerGraphSAGE(feat_dim, hidden_dim, emb_dim)
# encoder = OneLayerGraphSAGE(feat_dim, emb_dim)
mlp = MLPHead(emb_dim, emb_dim // 4, 40)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(mlp.parameters()), lr=1e-3)
epochs = 2000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
mlp = mlp.to(device)

In [12]:
data_2012_2013 = temp_partition_arxiv(data, year_bound=[-1,2012,2013], proportion=1.0)
data_2012_2013 = data_2012_2013.to(device)

In [13]:
best_acc = 0
best_encoder = None
best_mlp = None
evaluator = Evaluator(name='ogbn-arxiv')
for e in range(1, epochs + 1):
    train_loss = train(encoder, mlp, optimizer, data_2012_2013)
    val_loss, val_acc = test(encoder, mlp, data_2012_2013, evaluator)
    print(f"Epoch:{e}/{epochs} Train Loss:{round(train_loss,4)} Val Loss:{round(val_loss,4)} Val Acc:{round(val_acc, 4)}")
    if val_acc > best_acc:
        best_acc = val_acc
        best_encoder = deepcopy(encoder)
        best_mlp = deepcopy(mlp)

Epoch:1/2000 Train Loss:3.6866 Val Loss:3.6296 Val Acc:0.2034
Epoch:2/2000 Train Loss:3.6045 Val Loss:3.566 Val Acc:0.2045
Epoch:3/2000 Train Loss:3.5244 Val Loss:3.5053 Val Acc:0.2045
Epoch:4/2000 Train Loss:3.445 Val Loss:3.4446 Val Acc:0.2045
Epoch:5/2000 Train Loss:3.3669 Val Loss:3.3839 Val Acc:0.2045
Epoch:6/2000 Train Loss:3.2874 Val Loss:3.3274 Val Acc:0.2045
Epoch:7/2000 Train Loss:3.2117 Val Loss:3.2802 Val Acc:0.2045
Epoch:8/2000 Train Loss:3.1464 Val Loss:3.2567 Val Acc:0.2045
Epoch:9/2000 Train Loss:3.0974 Val Loss:3.2465 Val Acc:0.2045
Epoch:10/2000 Train Loss:3.0664 Val Loss:3.2367 Val Acc:0.2045
Epoch:11/2000 Train Loss:3.0481 Val Loss:3.2284 Val Acc:0.2045
Epoch:12/2000 Train Loss:3.0316 Val Loss:3.2009 Val Acc:0.2045
Epoch:13/2000 Train Loss:3.0069 Val Loss:3.1707 Val Acc:0.2045
Epoch:14/2000 Train Loss:2.978 Val Loss:3.1286 Val Acc:0.2047
Epoch:15/2000 Train Loss:2.9441 Val Loss:3.0987 Val Acc:0.2047
Epoch:16/2000 Train Loss:2.9196 Val Loss:3.0695 Val Acc:0.207
Epoch

Epoch:162/2000 Train Loss:1.5446 Val Loss:1.7345 Val Acc:0.508
Epoch:163/2000 Train Loss:1.5455 Val Loss:1.7313 Val Acc:0.5091
Epoch:164/2000 Train Loss:1.542 Val Loss:1.7219 Val Acc:0.5055
Epoch:165/2000 Train Loss:1.5405 Val Loss:1.7276 Val Acc:0.5127
Epoch:166/2000 Train Loss:1.5371 Val Loss:1.7261 Val Acc:0.5114
Epoch:167/2000 Train Loss:1.5371 Val Loss:1.7182 Val Acc:0.5152
Epoch:168/2000 Train Loss:1.5318 Val Loss:1.7237 Val Acc:0.5103
Epoch:169/2000 Train Loss:1.5275 Val Loss:1.711 Val Acc:0.5161
Epoch:170/2000 Train Loss:1.5264 Val Loss:1.7092 Val Acc:0.5122
Epoch:171/2000 Train Loss:1.5223 Val Loss:1.7099 Val Acc:0.5083
Epoch:172/2000 Train Loss:1.5228 Val Loss:1.7124 Val Acc:0.5131
Epoch:173/2000 Train Loss:1.517 Val Loss:1.7081 Val Acc:0.5152
Epoch:174/2000 Train Loss:1.5168 Val Loss:1.7075 Val Acc:0.515
Epoch:175/2000 Train Loss:1.5199 Val Loss:1.7081 Val Acc:0.5156
Epoch:176/2000 Train Loss:1.5083 Val Loss:1.7012 Val Acc:0.5172
Epoch:177/2000 Train Loss:1.5087 Val Loss:1.7

Epoch:294/2000 Train Loss:1.3256 Val Loss:1.6044 Val Acc:0.5517
Epoch:295/2000 Train Loss:1.3251 Val Loss:1.5883 Val Acc:0.5514
Epoch:296/2000 Train Loss:1.3225 Val Loss:1.5909 Val Acc:0.5487
Epoch:297/2000 Train Loss:1.3242 Val Loss:1.594 Val Acc:0.5472
Epoch:298/2000 Train Loss:1.3203 Val Loss:1.6032 Val Acc:0.545
Epoch:299/2000 Train Loss:1.3192 Val Loss:1.59 Val Acc:0.5492
Epoch:300/2000 Train Loss:1.3172 Val Loss:1.5967 Val Acc:0.5448
Epoch:301/2000 Train Loss:1.3202 Val Loss:1.6007 Val Acc:0.5459
Epoch:302/2000 Train Loss:1.3187 Val Loss:1.5897 Val Acc:0.5476
Epoch:303/2000 Train Loss:1.3142 Val Loss:1.5882 Val Acc:0.5484
Epoch:304/2000 Train Loss:1.3189 Val Loss:1.5806 Val Acc:0.5518
Epoch:305/2000 Train Loss:1.3175 Val Loss:1.5909 Val Acc:0.5486
Epoch:306/2000 Train Loss:1.3146 Val Loss:1.5878 Val Acc:0.5489
Epoch:307/2000 Train Loss:1.3151 Val Loss:1.5921 Val Acc:0.5489
Epoch:308/2000 Train Loss:1.3097 Val Loss:1.5848 Val Acc:0.552
Epoch:309/2000 Train Loss:1.3073 Val Loss:1.5

Epoch:430/2000 Train Loss:1.213 Val Loss:1.5488 Val Acc:0.5591
Epoch:431/2000 Train Loss:1.2082 Val Loss:1.5536 Val Acc:0.5593
Epoch:432/2000 Train Loss:1.2089 Val Loss:1.5611 Val Acc:0.5594
Epoch:433/2000 Train Loss:1.2071 Val Loss:1.558 Val Acc:0.5546
Epoch:434/2000 Train Loss:1.2051 Val Loss:1.5629 Val Acc:0.5562
Epoch:435/2000 Train Loss:1.2078 Val Loss:1.5571 Val Acc:0.5574
Epoch:436/2000 Train Loss:1.2073 Val Loss:1.5545 Val Acc:0.5593
Epoch:437/2000 Train Loss:1.2045 Val Loss:1.5559 Val Acc:0.5562
Epoch:438/2000 Train Loss:1.2069 Val Loss:1.5508 Val Acc:0.561
Epoch:439/2000 Train Loss:1.2066 Val Loss:1.5628 Val Acc:0.5546
Epoch:440/2000 Train Loss:1.2047 Val Loss:1.556 Val Acc:0.5518
Epoch:441/2000 Train Loss:1.2077 Val Loss:1.565 Val Acc:0.5556
Epoch:442/2000 Train Loss:1.2059 Val Loss:1.5679 Val Acc:0.5556
Epoch:443/2000 Train Loss:1.2052 Val Loss:1.5631 Val Acc:0.5528
Epoch:444/2000 Train Loss:1.2035 Val Loss:1.5666 Val Acc:0.5524
Epoch:445/2000 Train Loss:1.2031 Val Loss:1.5

Epoch:565/2000 Train Loss:1.1367 Val Loss:1.5621 Val Acc:0.557
Epoch:566/2000 Train Loss:1.133 Val Loss:1.5573 Val Acc:0.561
Epoch:567/2000 Train Loss:1.1312 Val Loss:1.5551 Val Acc:0.5587
Epoch:568/2000 Train Loss:1.1319 Val Loss:1.5601 Val Acc:0.5624
Epoch:569/2000 Train Loss:1.1348 Val Loss:1.5647 Val Acc:0.5554
Epoch:570/2000 Train Loss:1.1322 Val Loss:1.5577 Val Acc:0.5598
Epoch:571/2000 Train Loss:1.1294 Val Loss:1.5724 Val Acc:0.5538
Epoch:572/2000 Train Loss:1.1302 Val Loss:1.5579 Val Acc:0.5605
Epoch:573/2000 Train Loss:1.1285 Val Loss:1.5629 Val Acc:0.5563
Epoch:574/2000 Train Loss:1.1312 Val Loss:1.5618 Val Acc:0.5599
Epoch:575/2000 Train Loss:1.1269 Val Loss:1.5683 Val Acc:0.5571
Epoch:576/2000 Train Loss:1.1287 Val Loss:1.5576 Val Acc:0.5638
Epoch:577/2000 Train Loss:1.1301 Val Loss:1.563 Val Acc:0.5593
Epoch:578/2000 Train Loss:1.1286 Val Loss:1.5708 Val Acc:0.5571
Epoch:579/2000 Train Loss:1.1258 Val Loss:1.5528 Val Acc:0.561
Epoch:580/2000 Train Loss:1.1295 Val Loss:1.5

...

Epoch:1388/2000 Train Loss:0.7067 Val Loss:1.8733 Val Acc:0.5451
Epoch:1389/2000 Train Loss:0.6986 Val Loss:1.8572 Val Acc:0.5441
Epoch:1390/2000 Train Loss:0.6986 Val Loss:1.8708 Val Acc:0.5431
Epoch:1391/2000 Train Loss:0.702 Val Loss:1.8726 Val Acc:0.5431
Epoch:1392/2000 Train Loss:0.696 Val Loss:1.8634 Val Acc:0.5441
Epoch:1393/2000 Train Loss:0.6974 Val Loss:1.8511 Val Acc:0.5439
Epoch:1394/2000 Train Loss:0.701 Val Loss:1.8622 Val Acc:0.5483
Epoch:1395/2000 Train Loss:0.6979 Val Loss:1.8648 Val Acc:0.5456
Epoch:1396/2000 Train Loss:0.6919 Val Loss:1.8684 Val Acc:0.5425
Epoch:1397/2000 Train Loss:0.6962 Val Loss:1.8825 Val Acc:0.5431
Epoch:1398/2000 Train Loss:0.6887 Val Loss:1.8675 Val Acc:0.5448
Epoch:1399/2000 Train Loss:0.6931 Val Loss:1.8734 Val Acc:0.5405
Epoch:1400/2000 Train Loss:0.6921 Val Loss:1.8808 Val Acc:0.5427
Epoch:1401/2000 Train Loss:0.6932 Val Loss:1.8612 Val Acc:0.5436
Epoch:1402/2000 Train Loss:0.6949 Val Loss:1.8653 Val Acc:0.5414
Epoch:1403/2000 Train Loss:0

Epoch:1522/2000 Train Loss:0.6335 Val Loss:1.9735 Val Acc:0.5397
Epoch:1523/2000 Train Loss:0.6347 Val Loss:1.9672 Val Acc:0.5313
Epoch:1524/2000 Train Loss:0.6292 Val Loss:1.966 Val Acc:0.543
Epoch:1525/2000 Train Loss:0.6317 Val Loss:1.9612 Val Acc:0.5382
Epoch:1526/2000 Train Loss:0.6221 Val Loss:1.96 Val Acc:0.5364
Epoch:1527/2000 Train Loss:0.6358 Val Loss:1.9509 Val Acc:0.5409
Epoch:1528/2000 Train Loss:0.6295 Val Loss:1.9527 Val Acc:0.5388
Epoch:1529/2000 Train Loss:0.6266 Val Loss:1.9684 Val Acc:0.5374
Epoch:1530/2000 Train Loss:0.632 Val Loss:1.9792 Val Acc:0.5366
Epoch:1531/2000 Train Loss:0.6338 Val Loss:1.9494 Val Acc:0.5349
Epoch:1532/2000 Train Loss:0.6271 Val Loss:1.9635 Val Acc:0.5369
Epoch:1533/2000 Train Loss:0.6307 Val Loss:1.9669 Val Acc:0.5399
Epoch:1534/2000 Train Loss:0.6246 Val Loss:1.9686 Val Acc:0.5391
Epoch:1535/2000 Train Loss:0.6287 Val Loss:1.9743 Val Acc:0.5375
Epoch:1536/2000 Train Loss:0.628 Val Loss:1.9592 Val Acc:0.5422
Epoch:1537/2000 Train Loss:0.62

Epoch:1658/2000 Train Loss:0.5733 Val Loss:2.054 Val Acc:0.5327
Epoch:1659/2000 Train Loss:0.5725 Val Loss:2.054 Val Acc:0.5352
Epoch:1660/2000 Train Loss:0.5664 Val Loss:2.0631 Val Acc:0.5388
Epoch:1661/2000 Train Loss:0.5664 Val Loss:2.0587 Val Acc:0.5355
Epoch:1662/2000 Train Loss:0.5653 Val Loss:2.0601 Val Acc:0.54
Epoch:1663/2000 Train Loss:0.5642 Val Loss:2.0508 Val Acc:0.5363
Epoch:1664/2000 Train Loss:0.5631 Val Loss:2.0557 Val Acc:0.5372
Epoch:1665/2000 Train Loss:0.5707 Val Loss:2.068 Val Acc:0.531
Epoch:1666/2000 Train Loss:0.5692 Val Loss:2.0641 Val Acc:0.5335
Epoch:1667/2000 Train Loss:0.5644 Val Loss:2.0641 Val Acc:0.5361
Epoch:1668/2000 Train Loss:0.5657 Val Loss:2.0739 Val Acc:0.5333
Epoch:1669/2000 Train Loss:0.5704 Val Loss:2.0706 Val Acc:0.5375
Epoch:1670/2000 Train Loss:0.5676 Val Loss:2.0674 Val Acc:0.535
Epoch:1671/2000 Train Loss:0.5635 Val Loss:2.0568 Val Acc:0.5408
Epoch:1672/2000 Train Loss:0.567 Val Loss:2.073 Val Acc:0.533
Epoch:1673/2000 Train Loss:0.5592 V

Epoch:1793/2000 Train Loss:0.5077 Val Loss:2.189 Val Acc:0.531
Epoch:1794/2000 Train Loss:0.5136 Val Loss:2.1626 Val Acc:0.5369
Epoch:1795/2000 Train Loss:0.5071 Val Loss:2.1498 Val Acc:0.5363
Epoch:1796/2000 Train Loss:0.5116 Val Loss:2.1617 Val Acc:0.5369
Epoch:1797/2000 Train Loss:0.5029 Val Loss:2.1769 Val Acc:0.5299
Epoch:1798/2000 Train Loss:0.5045 Val Loss:2.179 Val Acc:0.5319
Epoch:1799/2000 Train Loss:0.5019 Val Loss:2.1989 Val Acc:0.531
Epoch:1800/2000 Train Loss:0.5146 Val Loss:2.1785 Val Acc:0.5354
Epoch:1801/2000 Train Loss:0.5084 Val Loss:2.1675 Val Acc:0.5358
Epoch:1802/2000 Train Loss:0.5011 Val Loss:2.1673 Val Acc:0.5333
Epoch:1803/2000 Train Loss:0.5035 Val Loss:2.1472 Val Acc:0.5308
Epoch:1804/2000 Train Loss:0.5081 Val Loss:2.1803 Val Acc:0.5249
Epoch:1805/2000 Train Loss:0.5097 Val Loss:2.1883 Val Acc:0.5251
Epoch:1806/2000 Train Loss:0.4995 Val Loss:2.1771 Val Acc:0.5307
Epoch:1807/2000 Train Loss:0.5079 Val Loss:2.1765 Val Acc:0.5354
Epoch:1808/2000 Train Loss:0.

Epoch:1929/2000 Train Loss:0.4595 Val Loss:2.3122 Val Acc:0.5347
Epoch:1930/2000 Train Loss:0.4544 Val Loss:2.2856 Val Acc:0.527
Epoch:1931/2000 Train Loss:0.4602 Val Loss:2.2937 Val Acc:0.5256
Epoch:1932/2000 Train Loss:0.4643 Val Loss:2.2886 Val Acc:0.5291
Epoch:1933/2000 Train Loss:0.4534 Val Loss:2.2778 Val Acc:0.5312
Epoch:1934/2000 Train Loss:0.4518 Val Loss:2.2936 Val Acc:0.5276
Epoch:1935/2000 Train Loss:0.4582 Val Loss:2.3057 Val Acc:0.5305
Epoch:1936/2000 Train Loss:0.4569 Val Loss:2.2869 Val Acc:0.528
Epoch:1937/2000 Train Loss:0.4557 Val Loss:2.2975 Val Acc:0.5277
Epoch:1938/2000 Train Loss:0.4543 Val Loss:2.2947 Val Acc:0.5288
Epoch:1939/2000 Train Loss:0.4569 Val Loss:2.31 Val Acc:0.5226
Epoch:1940/2000 Train Loss:0.4541 Val Loss:2.3081 Val Acc:0.5282
Epoch:1941/2000 Train Loss:0.4532 Val Loss:2.3171 Val Acc:0.5333
Epoch:1942/2000 Train Loss:0.4564 Val Loss:2.2992 Val Acc:0.5316
Epoch:1943/2000 Train Loss:0.4556 Val Loss:2.3115 Val Acc:0.531
Epoch:1944/2000 Train Loss:0.4

In [14]:
best_acc

0.5676767676767677

In [15]:
test_acc_list = []

## 2013-2014 ##

In [16]:
data_2013_2015 = temp_partition_arxiv(data, year_bound=[-1,2013,2015], proportion=1.0)
data_2013_2015 = data_2013_2015.to(device)

In [17]:
val_loss, val_acc = test(best_encoder, best_mlp, data_2013_2015, evaluator)
test_acc_list.append(val_acc)

In [18]:
val_loss, val_acc

(1.5454697608947754, 0.5645207935681648)

## 2015-2016 ##

In [19]:
data_2015_2017 = temp_partition_arxiv(data, year_bound=[-1,2015,2017], proportion=1.0)
data_2015_2017 = data_2015_2017.to(device)

In [20]:
val_loss, val_acc = test(best_encoder, best_mlp, data_2015_2017, evaluator)
test_acc_list.append(val_acc)

In [21]:
val_loss, val_acc

(1.6257272958755493, 0.533692817368013)

## 2017-2018 ##

In [22]:
data_2017_2019 = temp_partition_arxiv(data, year_bound=[-1,2017,2019], proportion=1.0)
data_2017_2019 = data_2017_2019.to(device)

In [23]:
val_loss, val_acc = test(best_encoder, best_mlp, data_2017_2019, evaluator)
test_acc_list.append(val_acc)

In [24]:
val_loss, val_acc

(1.6715991497039795, 0.5220428953377179)

## 2019-2020 ##

In [25]:
data_2019_2021 = temp_partition_arxiv(data, year_bound=[-1,2019,2021], proportion=1.0)
data_2019_2021 = data_2019_2021.to(device)

In [26]:
val_loss, val_acc = test(best_encoder, best_mlp, data_2019_2021, evaluator)
test_acc_list.append(val_acc)

In [27]:
val_loss, val_acc

(1.6447997093200684, 0.51965928029134)

In [28]:
print(test_acc_list)

[0.5645207935681648, 0.533692817368013, 0.5220428953377179, 0.51965928029134]


In [29]:
print(sum(test_acc_list) / len(test_acc_list))

0.5349789466413089
